In [2]:
# Import statements and setup matplotlib for jupyter
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [3]:
# Load data into a dataframe
url = 'https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-with-R-Third-Edition/master/Chapter07/letterdata.csv'
df = pd.read_csv(url)

NameError: name 'pd' is not defined

In [ ]:
# class Model(nn.Module):
#     # Create ANN architecture. 16 input nodes, 3 hidden layers of 50 nodes, output player of 26 nodes
#     def __init__(self, input_features=16, h1=50, h2=50, h3=50, h4=50, output_features=26):
#         super().__init__()
#         self.fc1 = nn.Linear(input_features, h1)
#         self.fc2 = nn.Linear(h1, h2)
#         self.fc3 = nn.Linear(h2, h3)
#         self.fc4 = nn.Linear(h3, h4)
#         self.out = nn.Linear(h4, output_features)

#     #Feed forward function
#     def forward(self, x):
#         # Using the exponential linear unit activation function
#         x = F.elu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.elu(self.fc3(x))
#         x = F.relu(self.fc4(x))
#         x = self.out(x)
#         return x

In [ ]:
class Model(nn.Module):
    # Create ANN architecture. 16 input nodes, 3 hidden layers of 50 nodes, output player of 26 nodes
    def __init__(self, input_features=16, h1=60, h2=60, output_features=26):
        super().__init__()
        self.fc1 = nn.Linear(input_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, output_features)

    #Feed forward function
    def forward(self, x):
        # Using the exponential linear unit activation function
        x = F.elu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [ ]:
# Set seed for reproducible results
torch.manual_seed(100)
model = Model()

In [ ]:
# Preview data format
df.head()

In [ ]:
#Scale all numeric values into a small range. We will use min-max scaling.
# df_numeric = df.drop(columns=["letter"])
# min_max_scalar = preprocessing.MinMaxScaler()
# df_numeric_minmax = min_max_scalar.fit_transform(df_numeric)
# df_numeric_minmax = pd.DataFrame(df_numeric_minmax)
# df_numeric_minmax.insert(0, "letter", df["letter"])
# df = df_numeric_minmax

# df_normalized = df.drop(columns=['letter'])
# df_normalized = preprocessing.normalize(df_normalized, norm='l2')
# df_normalized = pd.DataFrame(df_normalized)
# df_normalized.insert(0, "letter", df["letter"])
# df = df_normalized
# df

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df_dropped = df.drop(columns=['letter'])
# print(scaler.fit(df_dropped))
# print(scaler.mean_)
# print(scaler.transform(df_dropped))
# df_dropped = pd.DataFrame(scaler.transform(df_dropped))
# df_dropped.insert(0, "letter", df["letter"])
# df = df_dropped

In [ ]:
# Replace each character in the letter column with a numeric value for compatability
chars=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
i = 0.0
for j in chars:
    df['letter'] = df['letter'].replace(j, i)
    i +=1
df.head()

In [ ]:
# Seperate predictors (X) and labels (y) 
X = df.drop('letter', axis=1)
y = df['letter']

X = X.values
y = y.values

In [ ]:
# Split data into an 80% training set and 20% testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=800)

# Set tensor accuracy
# In the future set up for GPU processing using cuda
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
# Measure the error and select optimization method with a learning rate of 0.001
# Higher lr of 0.05 was resulting in overfitting at high epochs
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Train the model
epochs = 3000
losses = []
for i in range(epochs):
    y_pred = model.forward(X_train) 
    #Measure error
    loss = criterion(y_pred, y_train) 
    losses.append(loss.detach().numpy())

    # Setup back propagation for tuning weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Graph the rate of error at each epoch
plt.plot(range(epochs), losses)
plt.ylabel("Error")
plt.xlabel('Epoch')

In [ ]:
# Calculate accuracy
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        # Make predictions using test data
        y_val = model.forward(data)

        # Check how many are correct
        if y_val.argmax().item() == y_test[i]:
            correct += 1
percent = correct/y_test.shape[0] * 100
print(f'{correct} / {y_test.shape[0]} = {percent}%')

In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = model.forward(data)

        #Letter classifier prediction
        print(f'{i+1}.) {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

        #Correct or not
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'We got {correct} correct')

In [ ]:
print(chars[14])

In [ ]:
print(chars[6])